## Let's build GPT: from scratch, in code, spelled out

In [2]:
with open("input.txt") as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

In [3]:
# set up encoders and decoders

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [
    stoi[c] for c in s
]  # encode - take a string and output a list of integers
decode = lambda s: "".join([itos[c] for c in s])

In [4]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [5]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# train on randomnly sample 'chunks' of training data at once
block_size = 8
train_data[: block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target is: {target}")

tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])
When input is tensor([18]) the target is: 47
When input is tensor([18, 47]) the target is: 56
When input is tensor([18, 47, 56]) the target is: 57
When input is tensor([18, 47, 56, 57]) the target is: 58
When input is tensor([18, 47, 56, 57, 58]) the target is: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [16]:
torch.manual_seed(1337) # leet
batch_size = 4 # how many independent sequences to parallel process
block_size = 8 # maximum context length for predictions

def get_batch(split):
    # generate a small batch of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size + 1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print("inputs:")
print(xb)
print(xb.shape)
print("targets:")
print(yb)

print("-----")

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when the input is {context.tolist()}, the target is: {target}")
        

tensor([ 76049, 234249, 934904, 560986])
inputs:
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
targets:
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
when the input is [24], the target is: 43
when the input is [24, 43], the target is: 58
when the input is [24, 43, 58], the target is: 5
when the input is [24, 43, 58, 5], the target is: 57
when the input is [24, 43, 58, 5, 57], the target is: 1
when the input is [24, 43, 58, 5, 57, 1], the target is: 46
when the input is [24, 43, 58, 5, 57, 1, 46], the target is: 43
when the input is [24, 43, 58, 5, 57, 1, 46, 43], the target is: 39
when the input is [44], the target is: 53
when the input is [44, 53], the target is: 56
when the input is [44, 53, 56], the target is: 1
when

In [28]:
# Using the simplest possible neural network (in Andrej's opinion):
# BigramLanguageModel
# TODO: Go back to 'Make More' series for more in-depth tutorial on this model
# TODO: Go back to 'Make More' series regarding negative loss likelihood
import math
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embeddings_table = nn.Embedding(vocab_size, vocab_size) # embedding table of shape (vocab_size, vocab_size) = (65, 65)
        
    def forward(self, idx, targets = None):
        # idx and targets are both (B, T) tensors of 
        
        if targets is None:
            loss = None
        else:
            logits = self.token_embeddings_table(idx) # (B, T, C) = Batch by time by channel tensor - In our case, B = 4, T = 8, C = vocab_size
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # "How well are we correctly predicting the next character based on the logits?"
            
        return logits, loss
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(f"Current loss: {loss}, Expected loss: {-math.log(1/vocab_size)}") # negative log likelihood     
        

torch.Size([32, 65])
Current loss: 4.689231872558594, Expected loss: 4.174387269895637
